<a href="https://colab.research.google.com/github/darshanvjani/pytorch-lightning/blob/main/3.%20Lightning%20Trainer/simple_fc_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch_lightning
!pip install -U torch_xla
!pip install cloud-tpu-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages 

In [2]:
# import torch_xla as xla

In [3]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader,random_split
from tqdm import tqdm

import pytorch_lightning as pl
import torchmetrics
from torchmetrics import Metric

In [8]:
# from torch.nn.parallel.data_parallel import device
class NN(pl.LightningModule):
  def __init__(self, input_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size,50)
    self.fc2 = nn.Linear(50,num_classes)
    self.loss_fc = nn.CrossEntropyLoss()
    self.accuracy = torchmetrics.Accuracy()

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

  def _common_step(self, batch, batch_idx):
    x, y = batch
    x = x.reshape(x.size(0),-1)
    y_pred = self.forward(x)  # forward pass
    loss = self.loss_fc(y_pred, y)  # compute loss
    return loss, y_pred, y

  def training_step(self, batch, batch_idx):
    loss, y_pred, y = self._common_step(batch, batch_idx)
    self.log('train_loss', loss)
    return loss

  def test_step(self, batch, batch_idx):
    loss, y_pred, y = self._common_step(batch, batch_idx)
    self.log('test_loss', loss)
    return loss

  def validation_step(self, batch, batch_idx):
    loss, y_pred, y = self._common_step(batch, batch_idx)
    self.log('val_loss', loss)
    return loss

  def predict_step(self, batch, batch_idx):
    x, y = batch
    x = x.reshape(-1, 1)
    y_pred_final = self.forward(x)
    prediction = torch.argmax(y_pred_final, dim=1)
    return prediction

  def configure_optimizers(self):
    return optim.Adam(self.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#HP

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3


# Load Data
entire_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
train_ds, val_ds = random_split(entire_dataset, [50000, 10000])
test_ds = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)

In [9]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# for epoch in range(num_epochs):
#   for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):

#     # get data to cuda if possible
#     data = data.to(device=device)
#     targets = targets.to(device=device)

#     # get to correct shape
#     data = data.reshape(data.shape[0], -1)

#     #forward - loss calculation
#     scores = model.forward(data)
#     loss = criterion(scores, targets)

#     #zero-grad - backwards
#     optimizer.zero_grad()
#     loss.backward()

#     #gradient descent or adam step
#     optimizer.step()


trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=3, precision=16)
trainer.fit(model, train_loader, val_loader)
trainer.validate(model, val_loader)
trainer.test(model, test_loader)

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type             | Params
---------------------------------------------
0 | fc1     | Linear           | 39.2 K
1 | fc2     | Linear           | 510   
2 | loss_fc | CrossEntropyLoss | 0     
------------------------------

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.16542495787143707    │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.1581895351409912     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.1581895351409912}]

In [12]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples
